# Face Mask Detection (You-Only-Look-Once v8)


 ## Installation

 1. Create a new environment

```
python -m venv .venv
.venv/Scripts/activate # on windows
source .venv/bin/activate
```
2. Install Dependencies

```
pip install -r requirements.txt
```

3. Run app for training

```
python main.py  

```
or perform inference from the cli using,

```
yolo predict model='models\best.pt' source=' .mp4/.jpeg/etc'
```

## Summary
- Object Detection - Trained on YOLOv8n.pt (Model underfits, checkpoints not added till yet)
- Classification - Trained on YOLOv8n-cls.pt
- Mask, No Mask Detection and Classification.
- Custom Dataset (using CV). Labels corrected using OpenCV and observation.
- Trained on YOLOv8, locally for both classification and object-detection.
- Classification model available. Classes 0: Mask, 1: No-Mask
- Checkpoint for classification - models/best.pt', object detection - yet to converge.
- Helper functions for both classification and detection in utils/
- Output video directory -> output-vids/
  
## Roadmap

- More readability, more simplicity. Better code organisation.
- Better documentation.
- Deploy the model.
- Try to achieve better accuracy on the same dataset.
- Discuss with Senior CV Engineers on the exact same problem.


Please note: Please ignore the lack of proper formatting and enriching the colab document at the moment. Work under progress and under heavy-time constraints.

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!git clone https://github.com/Sh9hid/face-mask-detection.git

In [2]:
# Load your environment variables
csv_file = '/content/drive/MyDrive/data/data/train.csv'
images = '/content/drive/MyDrive/data/images'
videos = '/content/drive/MyDrive/data/videos'


In [ ]:
# !pip install -r requirements.txt


```
1. Preprocess: Clean Dataset.
```




```
# utils/clean.py
# from config.config import FILE_PATH, IMAGES_DIR

```



In [3]:
FILE_PATH = csv_file
IMAGES_DIR = images

In [5]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split


def create_clean_csv(data):
    """
    This function takes a dataset, cleans it, and saves it to a new CSV file.

    Steps:
    1. Handle duplicate entries
    2. Resolve conflicts by choosing 'face_with_mask' over others
    3. Categorize into relevant classnames
    4. Drop rows where 'classname' is None
    """

    # Exploratory Data Analysis
    unique_class_names = data['classname'].unique()

    # Step 1: Handle duplicate entries
    data = data.drop_duplicates()

    # Step 2: Resolve conflicts by choosing 'face_with_mask' over others
    def resolve_conflict(group):
        if 'face_with_mask' in group['classname'].values:
            return group[group['classname'] == 'face_with_mask']
        else:
            return group.iloc[:1]

    # Step 3: Categorize into relevant classnames
    def map_classnames(classname):
        if classname in ['face_with_mask', 'mask_colorful', 'mask_surgical',
                         'face_other_covering', 'scarf_bandana']:
            return 'mask'
        elif classname in ['face_no_mask']:
            return 'no_mask'

    # Apply map_classnames function
    data['classname'] = data['classname'].apply(map_classnames)

    # Apply resolve conflicts function and group by name
    data = data.groupby('name').apply(resolve_conflict).reset_index(drop=True)

    # Step 4: Drop rows where 'classname' is None (if any)
    data = data.dropna(subset=['classname'])

    # Save the cleaned dataset to a new CSV file
    output_path = 'data/cleaned_data.csv'
    data.to_csv(output_path, index=False)

    print(f"Data processed successfully and saved to {output_path}")


# Copy useful images from the dataset for training.
def process_images(data, IMAGES_DIR):
    """
    This function selects useful images from the entire-dataset for training.

    Steps:
    1. Create a list of images.
    2. Find useful images from the dataset for training.
    3. Copy useful images to /data/images for training.
    """

    # Step 1: Create a list of images.
    images_list = list(data['name'])

    # Step 2: Prepare the output directory for the images.
    IMAGES_OUTPUT_DIRECTORY_PATH = 'data/images'
    if not os.path.exists(IMAGES_OUTPUT_DIRECTORY_PATH):
        os.makedirs(IMAGES_OUTPUT_DIRECTORY_PATH)

    # Step 3: Copy useful images from the dataset to /data/images for training.
    for image in images_list:
        image_path = os.path.join(IMAGES_DIR, image)
        try:
            # Copy the image to the output directory.
            shutil.copy(image_path, IMAGES_OUTPUT_DIRECTORY_PATH)
        except Exception as e:
            # Handle any exceptions that occur during the copying process.
            print(f"Error processing {image}: {str(e)}")




```
2. Prepare training data: Labels and images
# utils/train.py
```



In [ ]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

def split_and_copy_images(annotation_path, images_path, output_path):
    # Load annotation data
    annotations = pd.read_csv(annotation_path)

    # Split data into training and validation sets
    train_df, val_df = train_test_split(annotations, test_size=0.2, random_state=42)

    TRAIN_IMAGES_DIR = os.path.join(output_path, 'train')
    VAL_IMAGES_DIR = os.path.join(output_path, 'val')

    # Get lists of images in each split
    train_images = list(train_df['name'])
    val_images = list(val_df['name'])

    # Create directories for training and validation data for Classification Model.
    directories = {
        'train_mask': os.path.join(TRAIN_IMAGES_DIR, 'mask'),
        'train_no_mask': os.path.join(TRAIN_IMAGES_DIR, 'no-mask'),
        'val_mask': os.path.join(VAL_IMAGES_DIR, 'mask'),
        'val_no_mask': os.path.join(VAL_IMAGES_DIR, 'no-mask')
    }

    # Create directories if they don't exist
    for dir_path in directories.values():
        os.makedirs(dir_path, exist_ok=True)

    # Process images
    for image_name in train_images + val_images:
        source_path = os.path.join(images_path, image_name)

        # Determine target directory based on split
        if image_name in train_images:
            split = 'train'
        elif image_name in val_images:
            split = 'val'
        else:
            print(f"Warning: Image {image_name} not found in train or val images.")
            continue

        # Determine class based on annotation
        found = False
        for index, row in annotations.iterrows():
            if row['name'] == image_name:
                found = True
                class_dir = directories[f'{split}_{row["classname"]}']
                break

        if not found:
            print(f"Warning: Image {image_name} not found in annotation.")
            continue

        target_path = os.path.join(class_dir, image_name)

        try:
            shutil.copy(source_path, target_path)
        except FileNotFoundError:
            print(f"Error: {source_path} not found.")

    print(f"Data split into training and validation sets and copied to {TRAIN_IMAGES_DIR} and {VAL_IMAGES_DIR} respectively.")




```
# 3. Train model
```



In [ ]:
!yolo classify train model='yolov8n-cls.pt' epochs=100 data='data/'

In [ ]:
# Verify installation
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/78.2 GB disk)


In [ ]:
# Load dataset from drive
# data = path/to/your/dataset  Else, follow the above code for loading data on YOLOv8

In [ ]:
# Train existing model, higher epochs to prevent underfitting. May have to decrease, if model starts overfitting
from ultralytics import YOLO
model = YOLO('yolov8n-cls.pt')
model.train(data=data, epochs=10)


100%|██████████| 5.30M/5.30M [00:00<00:00, 20.6MB/s]


Ultralytics YOLOv8.2.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/drive/MyDrive/data/data, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 6.23M/6.23M [00:00<00:00, 24.4MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/data/data/train... 131 images, 0 corrupt: 100%|██████████| 131/131 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/data/data/val... 22 images, 0 corrupt: 100%|██████████| 22/22 [00:00<?, ?it/s]


...

                   all      0.818          1



10 epochs completed in 0.121 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1437442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/MyDrive/data/data/train... found 131 images in 2 classes ✅ 
val: /content/drive/MyDrive/data/data/val... found 22 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 11.99it/s]


                   all      0.818          1
Speed: 0.3ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782a8844b5b0>
curves: []
curves_results: []
fitness: 0.9090909063816071
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8181818127632141, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9090909063816071}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.29229034077037463, 'inference': 0.9059039029208096, 'loss': 0.0006719069047407671, 'postprocess': 0.0006393952803178266}
task: 'classify'
top1: 0.8181818127632141
top5: 1.0

In [ ]:
# Save the model
!yolo export model=yolov8n-trained-onnx.pt format=onnx

/bin/bash: line 1: yolo: command not found


In [ ]:
# TODO : Add Inference loader
filepath = '/content/drive/MyDrive/data/data/val/mask/3720.png'

In [ ]:
results = model(video_path)




```
 Object Detection - utils\prepare_od.py and train_od.py
```



 TODO: Ask for better solution to the problem. I noticed a pattern of labels and went for a more brute force approach. Fill this document. \

In [ ]:
import os
import shutil
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split

def prepare_data(train_df, val_df):
    """
    Directories is a dictionary for creating objection detection
    preparation data as per YOLO format.

    """
    directories = {
        'TRAIN_IMAGES_DIR': 'data/data_od/images/train',
        'VAL_IMAGES_DIR': 'data/data_od/images/val',
        'TRAIN_LABELS_DIR': 'data/data_od/labels/train',
        'VAL_LABELS_DIR': 'data/data_od/labels/val'
    }

    for dir in directories.values():
        os.makedirs(dir, exist_ok=True)

    for index, row in train_df.iterrows():
        filename = row['name']
        shutil.copy(f'data/images/{filename}', directories['TRAIN_IMAGES_DIR'])

    for index, row in val_df.iterrows():
        filename = row['name']
        shutil.copy(f'data/images/{filename}', directories['VAL_IMAGES_DIR'])

    return directories

def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.size

def fix_labels(image_width, image_height, boxes):
    """
    TODO : Write Docs

    """
    min_x1 = image_width
    min_y1 = image_height
    max_x2 = 0
    max_y2 = 0

    for x1, y1, x2, y2, label in boxes:
        min_x1 = min(min_x1, x1)
        min_y1 = min(min_y1, y1)
        max_x2 = max(max_x2, x2)
        max_y2 = max(max_y2, y2)

    center_x = (min_x1 + max_x2) // 2
    center_y = (min_y1 + max_y2) // 2

    new_width = int(image_width * 0.8)
    new_height = int(image_height * 0.8)

    new_x1 = max(0, center_x - new_width // 2)
    new_y1 = max(0, center_y - new_height // 2)
    new_x2 = min(image_width, center_x + new_width // 2)
    new_y2 = min(image_height, center_y + new_height // 2)

    return [(new_x1, new_y1, new_x2, new_y2, label)]

def create_yolo_labels(df, dir):
    """
    TODO : Write Docs

    """
    for index, row in df.iterrows():
        filename = row['name']
        image_path = os.path.join('data/images', f'{filename}')

        try:
            img_width, img_height = get_image_dimensions(image_path)
        except Exception as e:
            print(f"Error opening image {filename}: {e}. Skipping...")
            continue

        x1, x2, y1, y2 = row['x1'], row['x2'], row['y1'], row['y2']

        boxes = [(x1, y1, x2, y2, 0 if row['classname'] == 'mask' else 1)]
        boxes = fix_labels(img_width, img_height, boxes)

        for new_x1, new_y1, new_x2, new_y2, class_id in boxes:
            new_width = new_x2 - new_x1
            new_height = new_y2 - new_y1

            new_x_center = (new_x1 + new_x2) / 2.0
            new_y_center = (new_y1 + new_y2) / 2.0

            new_x_center /= img_width
            new_y_center /= img_height
            new_width /= img_width
            new_height /= img_height

            if not (0 <= new_x_center <= 1 and 0 <= new_y_center <= 1 and 0 <= new_width <= 1 and 0 <= new_height <= 1):
                print(f"Invalid normalized coordinates for {filename}. Skipping...")
                continue

            label_path = os.path.join(dir, f'{filename[:4]}.txt')
            with open(label_path, 'w') as f:
                f.write(f'{class_id} {new_x_center} {new_y_center} {new_width} {new_height}\n')


annotations = pd.read_csv(FILE_PATH)

# Split data into training and validation sets
train_df, val_df = train_test_split(annotations, test_size=0.2, random_state=42)

directories = prepare_data(train_df, val_df)

create_yolo_labels(train_df, directories['TRAIN_LABELS_DIR'])
create_yolo_labels(val_df, directories['VAL_LABELS_DIR'])


In [ ]:
# yolo train object detection model
!yolo model=yolov8n.pt data=mask_detection.yaml epochs=100



```
 Running Inference
```



In [ ]:
# classification model
!yolo classify predict model=yolov8n-cls-trained.pt source=path/to/vid

In [ ]:
# object detection model
!yolo detect predict model=yolov8n-trained.pt source=path/to/vid